# Week 5 - Vector Space Model (VSM) and Topic Modeling

Over the next weeks, we are going to re-implement Sherin's algorithm and apply it to the text data we've been working on last week! Here's our roadmap:

**Week 6 - vectorization and linear algebra**
6. Dampen: weight the frequency of words (1 + log[count])
7. Scale: Normalize weighted frequency of words
8. Direction: compute deviation vectors

**Week 7 - Clustering**
9. apply different unsupervised machine learning algorithms
    * figure out how many clusters we want to keep
    * inspect the results of the clustering algorithm

**Week 8 - Visualizing the results**
10. create visualizations to compare documents

# WEEK 5 - DATA CLEANING

## Step 1 - Data Retrieval

In [1]:
# using glob, find all the text files in the poe_complete_works folder
import glob

files = glob.glob('./poe_complete_works/*.txt')
print(files)

['./poe_complete_works/vol2.txt', './poe_complete_works/vol1.txt']


In [2]:
# get all the data from the text files into the "documents" list
# P.S. make sure you use the 'utf-8' encoding
documents = []

for filename in files: 
    with open (filename, "r", encoding='utf-8', errors='ignore') as f:
        documents.append(f.read())

In [3]:
# print the first 1000 characters of the first document to see what it 
# looks like (we'll use this as a sanity check below)
documents[0][:1500]

"THE WORKS OF EDGAR ALLAN POE\n\nVOLUME II\n\nThe Raven Edition\n\n[Redactor's Note--Some endnotes are by Poe and some were added by\nGriswold. In this volume the notes are at the end.]\n\nContents:\n\n     The Purloined Letter\n     The Thousand-and-Second Tale of Scheherezade\n     A Descent into the Maelstroem\n     Von Kempelen and his Discovery\n     Mesmeric Revelation\n     The Facts in the Case of M. Valdemar\n     The Black Cat\n     The Fall of the House of Usher\n     Silence--a Fable\n     The Masque of the Red Death\n     The Cask of Amontillado\n     The Imp of the Perverse\n     The Island of the Fay\n     The Assignation\n     The Pit and the Pendulum\n     The Premature Burial\n     The Domain of Arnheim\n     Landor's Cottage\n     William Wilson\n     The Tell-Tale Heart\n     Berenice\n     Eleonora\n\n\n\n\nTHE PURLOINED LETTER\n\n     Nil sapientiae odiosius acumine nimio.\n\n                                _Seneca_.\n\nAt Paris, just after dark one gusty evening 

## Step 2 - Data Cleaning

In [4]:
# only select the text that's between the first occurence of the 
# the word "abstract" and the last occurence of the word "reference"
# Optional: print the length of the string before and after, as a 
# sanity check
# HINT: https://stackoverflow.com/questions/14496006/finding-last-occurrence-of-substring-in-string-replacing-that
# read more about rfind: https://www.tutorialspoint.com/python/string_rfind.htm

# for i,doc in enumerate(documents):
#     print(len(documents[i]), end=' ')
#     # save the result
#     documents[i] = doc
#     # print the length of the resulting string
#     print(len(documents[i]))
    
# one liner:
# documents = [doc[doc.index('abstract'):doc.rfind('reference')] for doc in documents]

# this is unnessary for this data set

In [5]:
# replace carriage returns (i.e., "\n") with a white space
# check that the result looks okay by printing the 
# first 1000 characters of the 1st doc:

documents = [doc.replace('\n', ' ') for doc in documents]
print(documents[0][:1000])

THE WORKS OF EDGAR ALLAN POE  VOLUME II  The Raven Edition  [Redactor's Note--Some endnotes are by Poe and some were added by Griswold. In this volume the notes are at the end.]  Contents:       The Purloined Letter      The Thousand-and-Second Tale of Scheherezade      A Descent into the Maelstroem      Von Kempelen and his Discovery      Mesmeric Revelation      The Facts in the Case of M. Valdemar      The Black Cat      The Fall of the House of Usher      Silence--a Fable      The Masque of the Red Death      The Cask of Amontillado      The Imp of the Perverse      The Island of the Fay      The Assignation      The Pit and the Pendulum      The Premature Burial      The Domain of Arnheim      Landor's Cottage      William Wilson      The Tell-Tale Heart      Berenice      Eleonora     THE PURLOINED LETTER       Nil sapientiae odiosius acumine nimio.                                  _Seneca_.  At Paris, just after dark one gusty evening in the autumn of 18-, I was enjoying the two

In [6]:
# replace the punctation below by a white space
# check that the result looks okay 
# (e.g., by print the first 1000 characters of the 1st doc)

punctuation = ['.', '...', '!', '#', '"', '%', '$', "'", '&', ')', 
               '(', '+', '*', '-', ',', '/', '.', ';', ':', '=', 
               '<', '?', '>', '@', '",', '".', '[', ']', '\\', ',',
               '_', '^', '`', '{', '}', '|', '~', '−', '”', '“', '’']


# remove ponctuation
for i,doc in enumerate(documents): 
    for punc in punctuation: 
        doc = doc.replace(punc, ' ')
    documents[i] = doc
    
print(documents[0][:1000])

THE WORKS OF EDGAR ALLAN POE  VOLUME II  The Raven Edition   Redactor s Note  Some endnotes are by Poe and some were added by Griswold  In this volume the notes are at the end    Contents        The Purloined Letter      The Thousand and Second Tale of Scheherezade      A Descent into the Maelstroem      Von Kempelen and his Discovery      Mesmeric Revelation      The Facts in the Case of M  Valdemar      The Black Cat      The Fall of the House of Usher      Silence  a Fable      The Masque of the Red Death      The Cask of Amontillado      The Imp of the Perverse      The Island of the Fay      The Assignation      The Pit and the Pendulum      The Premature Burial      The Domain of Arnheim      Landor s Cottage      William Wilson      The Tell Tale Heart      Berenice      Eleonora     THE PURLOINED LETTER       Nil sapientiae odiosius acumine nimio                                    Seneca    At Paris  just after dark one gusty evening in the autumn of 18   I was enjoying the two

In [7]:
# remove numbers by either a white space or the word "number"
# again, print the first 1000 characters of the first document
# to check that you're doing the right thing
for i,doc in enumerate(documents): 
    for num in range(10):
        doc = doc.replace(str(num), '')
    documents[i] = doc

print(documents[1][:1000])

THE WORKS OF EDGAR ALLAN POE  IN FIVE VOLUMES   The Raven Edition     VOLUME I   Contents        Edgar Allan Poe  An Appreciation      Life of Poe  by James Russell Lowell      Death of Poe  by N  P  Willis      The Unparalleled Adventures of One Hans Pfaall      The Gold Bug      Four Beasts in One      The Murders in the Rue Morgue      The Mystery of Marie Rogt      The Balloon Hoax      MS  Found in a Bottle      The Oval Portrait     EDGAR ALLAN POE  AN APPRECIATION      Caught from some unhappy master whom unmerciful Disaster    Followed fast and followed faster till his songs one burden bore      Till the dirges of his Hope that melancholy burden bore          Of  never  never more    THIS stanza from  The Raven  was recommended by James Russell Lowell as an inscription upon the Baltimore monument which marks the resting place of Edgar Allan Poe  the most interesting and original figure in American letters  And  to signify that peculiar musical quality of Poe s genius which inth

In [8]:
# Remove the stop words below from our documents
# print the first 1000 characters of the first document
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
              'ourselves', 'you', 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his', 'himself', 'she', 
              'her', 'hers', 'herself', 'it', 'its', 'itself', 
              'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 
              'be', 'been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
              'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 
              'between', 'into', 'through', 'during', 'before', 
              'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 
              'further', 'then', 'once', 'here', 'there', 'when', 
              'where', 'why', 'how', 'all', 'any', 'both', 'each', 
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 
              'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
              'too', 'very', 's', 't', 'can', 'will', 
              'just', 'don', 'should', 'now']


# remove stop words
for i,doc in enumerate(documents):
    for stop_word in stop_words:
        doc = doc.replace(' ' + stop_word + ' ', ' ')
    documents[i] = doc

print(documents[0][:1000])

THE WORKS OF EDGAR ALLAN POE  VOLUME II  The Raven Edition   Redactor Note  Some endnotes Poe added Griswold  In volume notes end    Contents        The Purloined Letter      The Thousand Second Tale Scheherezade      A Descent Maelstroem      Von Kempelen Discovery      Mesmeric Revelation      The Facts Case M  Valdemar      The Black Cat      The Fall House Usher      Silence  Fable      The Masque Red Death      The Cask Amontillado      The Imp Perverse      The Island Fay      The Assignation      The Pit Pendulum      The Premature Burial      The Domain Arnheim      Landor Cottage      William Wilson      The Tell Tale Heart      Berenice      Eleonora     THE PURLOINED LETTER       Nil sapientiae odiosius acumine nimio                                    Seneca    At Paris  dark one gusty evening autumn    I enjoying twofold luxury meditation meerschaum  company friend C  Auguste Dupin  little back library  book closet  au troisieme  No    Rue Dunot  Faubourg St  Germain  For o

In [9]:
# remove words with one and two characters (e.g., 'd', 'er', etc.)
# print the first 1000 characters of the first document

for i,doc in enumerate(documents):  
    doc = [x for x in doc.split() if len(x) > 2]
    doc = " ".join(doc)
    documents[i] = doc

print(documents[0][:1000])

THE WORKS EDGAR ALLAN POE VOLUME The Raven Edition Redactor Note Some endnotes Poe added Griswold volume notes end Contents The Purloined Letter The Thousand Second Tale Scheherezade Descent Maelstroem Von Kempelen Discovery Mesmeric Revelation The Facts Case Valdemar The Black Cat The Fall House Usher Silence Fable The Masque Red Death The Cask Amontillado The Imp Perverse The Island Fay The Assignation The Pit Pendulum The Premature Burial The Domain Arnheim Landor Cottage William Wilson The Tell Tale Heart Berenice Eleonora THE PURLOINED LETTER Nil sapientiae odiosius acumine nimio Seneca Paris dark one gusty evening autumn enjoying twofold luxury meditation meerschaum company friend Auguste Dupin little back library book closet troisieme Rue Dunot Faubourg Germain For one hour least maintained profound silence casual observer might seemed intently exclusively occupied curling eddies smoke oppressed atmosphere chamber For however mentally discussing certain topics formed matter conv


### Putting it all together

In [10]:
# package all of your work above into a function that cleans a given document

def clean_list_of_documents(documents):
    
    cleaned_docs = []

    for i,doc in enumerate(documents):
        # only keep the text after the abstract
        # doc = doc[doc.index('abstract'):]
        # only keep the text before the references
        # doc = doc[:doc.rfind('reference')]
        # replace return carriage with white space
        doc = doc.replace('\n', ' ')
        # remove ponctuation
        for punc in punctuation: 
            doc = doc.replace(punc, ' ')
        # remove numbers
        for i in range(10):
            doc = doc.replace(str(i), ' ')
        # remove stop words
        for stop_word in stop_words:
            doc = doc.replace(' ' + stop_word + ' ', ' ')
        # remove single characters and stem the words 
        doc = [x for x in doc.split() if len(x) > 2]
        doc = " ".join(doc)
        # save the result to our list of documents
        cleaned_docs.append(doc)
        
    return cleaned_docs

In [11]:
# reimport your raw data
documents = []

for filename in files: 
    with open (filename, "r", encoding='utf-8', errors='ignore') as f:
        documents.append(f.read())
        
# clean your files using the function above
docs = clean_list_of_documents(documents)

# print the first 1000 characters of the first document
print(docs[0][:1000])

THE WORKS EDGAR ALLAN POE VOLUME The Raven Edition Redactor Note Some endnotes Poe added Griswold volume notes end Contents The Purloined Letter The Thousand Second Tale Scheherezade Descent Maelstroem Von Kempelen Discovery Mesmeric Revelation The Facts Case Valdemar The Black Cat The Fall House Usher Silence Fable The Masque Red Death The Cask Amontillado The Imp Perverse The Island Fay The Assignation The Pit Pendulum The Premature Burial The Domain Arnheim Landor Cottage William Wilson The Tell Tale Heart Berenice Eleonora THE PURLOINED LETTER Nil sapientiae odiosius acumine nimio Seneca Paris dark one gusty evening autumn enjoying twofold luxury meditation meerschaum company friend Auguste Dupin little back library book closet troisieme Rue Dunot Faubourg Germain For one hour least maintained profound silence casual observer might seemed intently exclusively occupied curling eddies smoke oppressed atmosphere chamber For however mentally discussing certain topics formed matter conv

## Step 3 - Build your list of vocabulary

This list of words (i.e., the vocabulary) is going to become the columns of your matrix.

In [12]:
import math
import numpy as np

In [13]:
# create a function that takes in a list of documents
# and returns a set of unique words. Make sure that you
# sort the list alphabetically before returning it. 

def get_vocabulary(docs):
    voc = []
    for doc in docs:
        for word in doc.split():
            if word not in voc: 
                voc.append(word)
    voc = list(set(voc))
    voc.sort()
    return voc

# Then print the length of your vocabulary (it should be 
# around 5500 words)
vocabulary = get_vocabulary(docs)
print(len(vocabulary))

14961


## Step 4 - transform your documents in to 100-words chunks

In [14]:
# create a function that takes in a list of documents
# and returns a list of 100-words chunk 
# (with a 25 words overlap between them)
# Optional: add two arguments, one for the number of words
# in each chunk, and one for the overlap

def flatten_and_overlap(docs, window_size=100, overlap=25):
    
    # create the list of overlapping documents
    new_list_of_documents = []
    
    # flatten everything into one string
    flat = ""
    for doc in docs:
        flat += doc
    
    # split into words
    flat = flat.split()

    # create chunks of 100 words
    high = window_size
    while high < len(flat):
        low = high - window_size
        new_list_of_documents.append(flat[low:high])
        high += overlap
    return new_list_of_documents

chunks = flatten_and_overlap(docs)

In [15]:
# create a for loop to double check that each chunk has 
# a length of 100
# Optional: use assert to do this check
for chunk in chunks: 
    assert(len(chunk) == 100)

# WEEK 6 - VECTOR MANIPULATION

## Step 5 - Create a word by document matrix

In [16]:
# 1) create an empty dataframe using pandas
# the number of rows should be the number of chunks we have
# the number of columns should be size of the vocabulary
import pandas as pd
df = pd.DataFrame(0, index=np.arange(len(chunks)), columns=vocabulary)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3684 entries, 0 to 3683
Columns: 14961 entries, AAEM to zusammen
dtypes: int64(14961)
memory usage: 420.5 MB


In [27]:
df

,AAEM,ADVENTURES,AFTER,ALCMAN,ALLAN,AMONTILLADO,AND,ANTIOCHUS,APPRECIATION,ARNHEIM,...,zeal,zealous,zenith,zig,zigzag,zodiacal,zone,zones,zufalle,zusammen
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# 2) fill out the dataframe with the count of words for each document
# (use two for loops to iterate through the documents and the vocabulary)
for i,chunk in enumerate(chunks):
    for word in chunk:
        if word in df.columns: 
            df.loc[i,word] += 1

In [29]:
# 3) Sanity check: make sure that your counts are correct
# (e.g., if you know that a words appears often in a document, check that
# the number is also high in your dataframe; and vice-versa for low counts)
df.loc[0,'raven']

0

In [19]:
# 4) Putting it together: create a function that takes a list of documents
# and a vocabulary as arguments, and returns a dataframe with the counts
# of words: 
def docs_by_words_df(chunks, vocabulary):
    df = pd.DataFrame(0, index=np.arange(len(chunks)), columns=vocabulary)
    
    # fill out the matrix with counts
    for i,chunk in enumerate(chunks):
        for word in chunk:
            if word in df.columns: 
                df.loc[i,word] += 1
            
    return df

# call the function and check that the resulting dataframe is correct
df = docs_by_words_df(chunks, vocabulary)
df.loc[0,'students']

0

## Step 6 - Weight word frequency

In [20]:
# 5) create a function that adds one to the current cell and takes its log
# IF the value in the cell is not zero
def one_plus_log(cell):
    if cell != 0: 
        return 1 + math.log(cell)
    else:
        return 0

In [21]:
# 6) use the "applymap" function of the dataframe to apply the function 
# above to each cell of the table
df_log = df.applymap(one_plus_log)

In [22]:
# 7) check that the numbers in the resulting matrix look accurate;
# print the value before and after applying the function above
print("before one + log: ", df.loc[0,'students'])
print("after one + log: ", 1 + math.log(df.loc[0,'students']))
print("Value in the dataframe: ", df_log.loc[0,'students'])

before one + log:  0


ValueError: math domain error

## Step 7 - Matrix normalization

In [ ]:
# 8) look at the image below; why do you think that we need to normalize our 
# data before clustering in this particular case? 

<img src="https://i.stack.imgur.com/N2unM.png" />

In general, it's common practice to normalize your data before clustering - so that variables are comparable.

In [ ]:
# 9) describe how the min-max normalization works:

<img src="https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/media/aml-normalization-minmax.png" />

In [ ]:
# 10) describe how normalizing using a z-score works:

<img src="https://cdn-images-1.medium.com/max/1600/1*13XKCXQc7eabfZbRzkvGvA.gif"/>

In [ ]:
# 11) describe how normalizing to unit norm works

Resources: 
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer
* http://mathworld.wolfram.com/NormalVector.html

We are going to work with some pre-made normalization functions from sklearn (feel free to skim this page):
* https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

In [ ]:
# 12) since we are working with vectors, apply the Normalizer from 
# sklearn.preprocessing to our dataframe. Print a few values 
# before and after to make sure you've applied the normalization
from sklearn.preprocessing import Normalizer

scaler = Normalizer()
df_log[df_log.columns] = scaler.fit_transform(df_log[df_log.columns])
df_log[df_log.columns[500:600]]

In [ ]:
# 13) create a function that takes a dataframe as argument and where a second
# argument is the type of normalization (MinMaxScaler, Normalizer, StandardScaler)
# and returns the normalized dataframe
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

def normalize_df(df, method='Normalizer'):
    
    # choose the normalization strategy
    scaler = None
    if method == 'Normalizer': scaler = Normalizer()
    if method == 'MinMaxScaler': scaler = MinMaxScaler()
    if method == 'StandardScaler': scaler = StandardScaler()
        
    # apply the normalization
    if scaler != None:
        df[df.columns] = scaler.fit_transform(df[df.columns])

    # return the resulting dataframe
    return df

## Step 8 - Deviation Vectors

<img src="https://www.dropbox.com/s/9f73r7pk7bi7vh9/deviation_vectors.png?dl=1" />

In [ ]:
# 14) compute the sum of the vectors
v_sum = np.sum(df_log.values, axis=0)

In [ ]:
# 15) normalize the vector (find its average)
def vector_length(u):
    return np.sqrt(np.dot(u, u))

def length_norm(u):
    return u / vector_length(u)

v_avg = length_norm(v_sum)

In [ ]:
# 16) take each vector and subtract its components along v_avg

matrix = df_log.values

for row in range(df_log.shape[0]):

    # this is one vector (row
    v_i = matrix[row,:]

    # we subtract its component along v_average
    scalar = np.dot(v_i,v_avg)
    sub = v_avg * scalar

    # we replace the row by the deviation vector
    matrix[row,:] = length_norm(v_i - sub)

In [ ]:
# 17) put the code above in a function that takes in a dataframe as an argument
# and computes deviation vectors of each row (=document)
def vector_length(u):
    return np.sqrt(np.dot(u, u))

def length_norm(u):
    return u / vector_length(u)

def transform_deviation_vectors(df):
    
    # get the numpy matrix from the df
    matrix = df.values
    
    # compute the sum of the vectors
    v_sum = np.sum(matrix, axis=0)
    
    # normalize this vector (find its average)
    v_avg = length_norm(v_sum)
    
    # we iterate through each vector
    for row in range(df_log.shape[0]):
        
        # this is one vector (row
        v_i = matrix[row,:]
        
        # we subtract its component along v_average
        scalar = np.dot(v_i,v_avg)
        sub = v_avg * scalar
        
        # we replace the row by the deviation vector
        matrix[row,:] = length_norm(v_i - sub)
    
    return df

In [ ]:
df = transform_deviation_vectors(df_log)

# WEEK 7 - CLUSTERING

## Step 9 - Clustering

### Figuring out how many clusters we should pick

1) Plot the inertia of kmeans using this example from datacamp: 
* https://campus.datacamp.com/courses/unsupervised-learning-in-python/clustering-for-dataset-exploration?ex=6

In [ ]:
# 1a) create a list of inertia values for k 1-10

from sklearn.cluster import KMeans
%matplotlib inline
import matplotlib.pyplot as plt

ks = range(1,11)
inertias = []

for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    # Fit model to df
    model.fit(df)
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)

In [ ]:
# 1b) plot the inertia values using matplotlib
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

1c) What can you conclude from the elbow method?

since there is no elbow KMeans is not appropriate for the data

2) Visualize your data using T-SNE
* https://campus.datacamp.com/courses/unsupervised-learning-in-python/visualization-with-hierarchical-clustering-and-t-sne?ex=11
* https://www.datacamp.com/community/tutorials/introduction-t-sne


In [ ]:
# 2a) plot the T-SNE graph using a learning rate of 200
from sklearn.manifold import TSNE

# Create a TSNE instance: model
model = TSNE(learning_rate=200)

# Apply fit_transform to samples: tsne_features
tsne_features = model.fit_transform(df)

# Select the 0th feature: xs
xs = tsne_features[:,0]

# Select the 1st feature: ys
ys = tsne_features[:,1]

# Scatter plot, coloring by variety_numbers
plt.scatter(xs, ys)
plt.show()

2b) What can you conclude from T-SNE?

That our data fits into about 17 clusters

Note: T-SNE is great, but there is also some controversy on how much you should trust this algorithm:
* [Shortcomings of T-SNE](https://stats.stackexchange.com/questions/270391/should-dimensionality-reduction-for-visualization-be-considered-a-closed-probl)
* [Limitations of T-SNE](https://stats.stackexchange.com/questions/263539/clustering-on-the-output-of-t-sne)

numpy.argsort()

### Kmeans

In [ ]:
# 3a) apply k-means to our data with k=10 and print the first 10 words
# that are the most associated with each cluster centroids
# Hint: look at the cluster_centers_ of the KMeans object to find the centroids

kmeans_results = KMeans(n_clusters=10).fit(df)

for n in range(kmeans_results.n_clusters):
    print("Cluster ", n, ": ", end=" ")
    
    centroid = kmeans_results.cluster_centers_[n]
    
    indicies = centroid.argsort()[-11:]
    
    for i in indicies:
        print(vocabulary[i], end=" ")
    
    print(" ")

3b) interpret the cluster above; do they make sense to you?

- cluster 0: words around the prompt and study subjects
- cluster 1: words around subjects and testing 
- cluster 2: words around data and analysis

### Hierarchical clustering

4) use hierarchical clustering on the data; feel free to refer to the datacamp lesson below: 
* https://campus.datacamp.com/courses/unsupervised-learning-in-python/visualization-with-hierarchical-clustering-and-t-sne?ex=3

In [ ]:
#4a) plot the dendogram using the link above (method = complete)

from scipy.cluster.hierarchy import linkage, dendrogram

# Calculate the linkage: mergings
mergings = linkage(df, method='complete')

# Plot the dendrogram, using words as labels
dendrogram(mergings,
           leaf_rotation=90,
)
plt.show()

4b) was the dendodram useful?

No; too many words

In [ ]:
#4c) we are going to use agglomerative clustering here 
# from the sklean library 

from sklearn.cluster import AgglomerativeClustering
import numpy as np

clustering = AgglomerativeClustering(n_clusters=6).fit(df)

print(clustering)

In [ ]:
# 4d) compute the center of the cluster
# unfortunately sklearn doesn't provide you with the centroids, but you can use the link below:
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html

from sklearn.neighbors.nearest_centroid import NearestCentroid

clf = NearestCentroid()
clf.fit(df, clustering.labels_)

In [ ]:
# 4e) print the top 10 words for each cluster centroid

centroids_words = [[] for i in range(6)]

for i in range(0,6):
    for j in range(1,11):
        n = np.argsort(clf.centroids_[i])[-j]
        centroids_words[i].append(df.columns[n])

for i in range(0,6):
    print(i,centroids_words[i])

4f) interpret the cluster above; do they make sense to you?

some of the clusters make sense and have common themes and some do not have an easily distinguishable common thread

### DBScan 

5) Use DBscan (with epsilon=5, min_samples=10) to cluster your data

In [ ]:
# 5a) apply DBScan on your data
# Hint: https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html

from sklearn.cluster import DBSCAN

db = DBSCAN(eps=1, min_samples=10).fit(df)
n_clusters_ = len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)

In [ ]:
# 5b) find the cluster centroid (using the code from question 4d)

clf_DBScan = NearestCentroid()
clf_DBScan.fit(df, db.labels_)

In [ ]:
# 5c) print the top ten words
centroids_words_DBScan = [[] for i in range(n_clusters_)]

for i in range(0,n_clusters_):
    for j in range(1,11):
        n = np.argsort(clf_DBScan.centroids_[i])[-j]
        centroids_words_DBScan[i].append(df.columns[n])

for i in range(0,n_clusters_):
    print(i,centroids_words_DBScan[i])

5d) How many clusters do you have? Do they make sense to you? Interpret them below. 

10 clusters

- cluster 0: words around students and learning
- cluster 1: words around student outcomes
- cluster 2: doesn't really have a common thread
- cluster 3: words around testing and scoring

### NMF

6) Use NMF to find topics in our dataset
* https://campus.datacamp.com/courses/unsupervised-learning-in-python/discovering-interpretable-features?ex=3

In [ ]:
# 6a) Use the code above to apply the NMF model

from sklearn.decomposition import NMF

# Create an NMF instance: model
model = NMF(n_components=6)

# Fit the model to df
model.fit(normalize_df(df, method='MinMaxScaler'))


In [ ]:
# 6b) print the top ten words of each component

num_clusters = 6
centroids_words_NMF = [[] for i in range(num_clusters)]

for i in range(0,num_clusters):
    for j in range(1,11):
        n = np.argsort(model.components_[i])[-j]
        centroids_words_NMF[i].append(df.columns[n])

for i in range(0,num_clusters):
    print(i,centroids_words_NMF[i])

6c) Interpret the cluster above; how do they compare to kmeans, hierarchical clustering and DBscan?

The groupings are similar but all methods produce slightly different clusters and words within those clusters. When less clusters are produced the words are harder to categorize. 

## Step 10 - Visualizing the results

## Final Step - Putting it all together: 

In [ ]:
# in python code, our goal is to recreate the steps above as functions
# so that we can just one line to run topic modeling on a list of 
# documents: 
def ExtractTopicsVSM(documents, numTopics):
    ''' this functions takes in a list of documents (strings), 
        runs topic modeling (as implemented by Sherin, 2013)
        and returns the clustering results, the matrix used 
        for clustering a visualization '''
    
    # step 2: clean up the documents
    documents = clean_list_of_documents(documents)
    
    # step 3: let's build the vocabulary of these docs
    vocabulary = get_vocabulary(documents)
    
    # step 4: we build our list of 100-words overlapping fragments
    documents = flatten_and_overlap(documents)
    
    # step 5: we convert the chunks into a matrix
    matrix = docs_by_words_matrix(documents, vocabulary)
    
    # step 6: we weight the frequency of words (count = 1 + log(count))
    matrix = one_plus_log_mat(matrix, documents, vocabulary)
    
    # step 7: we normalize the matrix
    matrix = normalize(matrix)
    
    # step 8: we compute deviation vectors
    matrix = transform_deviation_vectors(matrix, documents)
    
    # step 9: we apply a clustering algorithm to find topics
    results_clustering = cluster_matrix(matrix)
    
    # step 10: we create a visualization of the topics
    visualization = visualize_clusters(results_clustering, vocabulary)
    
    # finally, we return the clustering results, the matrix, and a visualization
    return results_clustering, matrix, visualization